# Introduction to ROOT package
Author: Marcin Konecki
# Example 0
type root in shell and try some simple calculations, draw simple TF1 function and browse a root file

# Example 1 - histogramming

In [ ]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-

import sys
from ROOT import *
from readData import readData 
import numpy as np

#
# read data from a text file
#
throw1 = []
throw2 = []
readData('data_dice.txt', throw1, throw2)

#
# print data 
#
allDice=throw1+throw2
sumDice=np.add(throw1,throw2)
print ('number of    throws1: {0:2d}, number of throws2: {1:2d}'.format(len(throw1),len(throw2)) )
print ('number of all throws: {0:2d}, number of combined throws: {1:2d}'.format(len(allDice),len(sumDice)) )

#
# define a Canva and histogram
#
c = TCanvas('cDice','cDice',1000,500)
c.Divide(2,1)
hDice = TH1D('hDice','All throws; dice; #events',6,0.5,6.5)
hDice.Sumw2()
hDiceSum = TH1D('hDiceSum','Throw 1 and 2 combined; sum of dice; #events',12,0.5,12.5)
hDiceSum.Sumw2()

# decorations 
hDice.SetMinimum(0) 
hDice.GetXaxis().SetNdivisions(106)
hDice.SetStats(0)
hDice.SetFillStyle(1)
#hDice.SetFillStyle(3444)
#hDice.SetFillColor(kGreen)
hDice.SetLineColor(kRed)
hDice.SetLineWidth(2)
hDice.SetLineStyle(1)
hDice.SetBarWidth(0.8)
hDice.SetBarOffset(0.1)
c.SetGrid(0,1)

for i in range(len(allDice)) :
  hDice.Fill( allDice[i])
for i in range(len(sumDice)) :
  hDiceSum.Fill( sumDice[i])

pad1=c.cd(1)
pad1.SetGrid(0,1)
hDice.DrawCopy('b histo')

c.cd(2)
hDiceSum.SetMinimum(0.)
hDiceSum.GetXaxis().SetTitle("\sum")
#fill bin x=1 with weight 4.
hDiceSum.Fill(1,4)
#for test of Sumw2() play below
#it seems that in current version of ROOT Sumw2() is switched on 
#irrespectively of settings, when filling with weight is done.
#to switch it off (do not do that!) uncomment libe below after filling, before plotting
#hDiceSum.Sumw2(kFALSE)

hDiceSum.DrawCopy('histo')
hDiceSum.DrawCopy('E same')
hDiceSum.Print("all")
print('-----------')
c.Draw()
c.Print('cDice.png')

#input('press enter to store histo and exit')
outfile=TFile.Open('histos.root',"RECREATE")
outfile.cd()
hDice.Write()
hDiceSum.Write()
outfile.Close()

#cleanup
hDice.Delete()
hDiceSum.Delete()
#c.Close()


# Example 2 - graphs
access to histograms, graphs, decorations

In [ ]:
#!/usr/bin/python3
from ROOT import *
f=TFile('histos.root')
f.ls()
hDice=f.Get('hDice')
#hDice.Print('all')

print("sum of entries: ",hDice.GetEntries())
print("integral:       ",hDice.Integral() )
hDice.Scale(1./hDice.Integral())
hDice.SetMarkerStyle(20)
hDice.SetMarkerColor(2)
hDice.SetMinimum(0.1)
hDice.SetMaximum(0.25)
print("integral:       ",hDice.Integral() )
nbins=hDice.GetNbinsX()

#acess to content bin-by-bin.
# bin==0 underflow
# bin=nbins+1 overflow
for i in range(1,nbins+1):
    print("bin %d [%5.2f,%5.2f[ = %4.3f"% (i,hDice.GetBinLowEdge(i),hDice.GetBinLowEdge(i+1),hDice.GetBinContent(i)))
    
gr=TGraphErrors(nbins)
gr.SetMarkerStyle(21)
gr.SetMarkerColor(3)
for i in range(nbins):
    gr.SetPoint(i,hDice.GetBinCenter(i+1), hDice.GetBinContent(i+1))
    gr.SetPointError(i, 0., hDice.GetBinError(i+1))

c=TCanvas('cDrawings','simple drawings',1200,500)
c.Divide(3,1)
c.cd(1)
hDice.DrawCopy('hist e')
c.cd(2)
gr.Draw('AC PX')
c.cd(3)
hNull=hDice.Clone()
hNull.SetTitle('Histo and Graph')
hNull.DrawCopy('hist')
gr.Draw('PZ')
c.Draw()
c.Print('cDrawings.png')
    

# Example 3 - functions

In [ ]:
#!/usr/bin/python3
import math
from ROOT import *

cFunctions=TCanvas('cFunctions')
f1=TF1("f1","[0]*x*sin([1]*x)",0.,10.)
f1.SetParameter(0,5.)
f1.SetParameter(1,1.)
f1.SetLineColor(kYellow)
f1.DrawCopy()

f2=TF1('f2','gaus(x,[0..2])*expo(x,[3..4])',0.,10.)
f2.SetParameter(0,0.1)
f2.SetParameter(1,1.)
f2.SetParameter(2,1.)
f2.SetParameter(3,3.)
f2.SetParameter(4,1.)
f2.SetLineColor(kRed)
f2.DrawCopy('same')

def mySin(x, par):
    return par[0]*sin(par[1]*x[0])

f3 = TF1('f3',mySin, 0., 10., 2)
f3.SetParameter(0, 20.);
f3.SetParameter(1, 0.5);
f3.SetLineColor(kBlue)
f3.DrawCopy('same')

legend=TLegend(0.1,25,4.,40.,"","")
legend.AddEntry(f1,'[0]*x*sin([1]*x)')
legend.AddEntry(f2,'gaus(0)*expo(3)')
legend.AddEntry(f3,'mySin')
legend.Draw()

cFunctions.Update()
cFunctions.Draw()
cFunctions.Print('.png')

# Example 4 - uniform random number generator

In [ ]:
#!/usr/bin/python3
import math
from ROOT import *

gRandom.SetSeed(2018)
def myDistr() :
  val = gRandom.Uniform(1.)
  if val < 0.5 : val -= 0.5 
  else: val += 0.5 
  return val

sigma = math.sqrt(7./12)
mu = 0.5

#def main() :
cFlat = TCanvas('cFlat','cFlat',800,600)
hVal = TH1D('hVal','flat*2',300, -1., 2.)
hVal.SetStats(0)

for i in  range(1000000):
  val=myDistr()
  hVal.Fill(val)

hVal.DrawCopy()
hVal.Delete()
cFlat.Draw()
cFlat.Print('.png')


In [ ]:
#!/usr/bin/python3

import sys
import math
from ROOT import *

from myDistr_2step import myDistr, sigma, mu 

cCTG = TCanvas('cCTG','cCTG',1000,500)
hCTG = TH1D('hCTG','hCTG',100, -5., 5.)
hVal = TH1D('hVal','hVal',300, -1., 2.)

#
# loop over experiments, each gives and average X_i
#
nDistrib = 50000 #
for idist in range (0,nDistrib) :


# single try X_i, repeated nProb
#
  sum = 0
  nProb = 25 #enough for average
  for proba in range(0,nProb) :
    val = myDistr()
    hVal.Fill(val)
    sum += val
  #
  # result for singe X_i computation is histogrammed
  #
  ctg_numerator = (sum/nProb)-mu  
  ctg_denominator = sigma/math.sqrt(nProb)
  ctg = ctg_numerator/ctg_denominator
  hCTG.Fill(ctg)


gStyle.SetOptStat(1110)
cCTG.Divide(2,1)
cCTG.cd(1)
gStyle.SetOptStat(1110)
hVal.DrawCopy()

cCTG.cd(2)
hCTG.Scale(1./nDistrib*10)    #10 due to bin whdth
hCTG.Draw('hist')

g1 = TF1("g1", "gaus",-5,5 )
sigmaGauss = 1.
g1.SetParameter(0, 1./sigmaGauss/math.sqrt(2*math.pi) )
g1.SetParameter(1,0)
g1.SetParameter(2,sigmaGauss)
g1.DrawCopy('same')
cCTG.Draw()
cCTG.Print("cCTG.png")


# Example 5 - event processing

In [ ]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
from ROOT import *
from array import array

## A C/C++ structure is required, to allow memory based access
gROOT.ProcessLine(
   "struct Vertex_t {\
    Int_t id;\
    Double_t x,y,z; \
    double perp() { return sqrt(x*x+y*y); } \
};")

gRandom.SetSeed(2022)
vertex = Vertex_t()
pid = array('i',[0])

f = TFile('myTree.root','RECREATE')
myTree  = TTree('myTree','simple tree with vertex data')
myTree.Branch('vertex',vertex)
myTree.Branch('pid',pid,'pid/I')


for i in range(1000) :
  pid[0] = gRandom.Integer(100)
  vertex.id = i
  vertex.x = gRandom.Uniform(10.)
  vertex.y = gRandom.Uniform(10.)
  vertex.z = gRandom.Gaus(10.,2.)
  
#  print ('pid:{0:2d}, vertex id:{1:3d}, perp={2:5.2f}'.format(pid[0],vertex.id,vertex.perp()))
  myTree.Fill()       
myTree.Print("all")
f.Write()
f.Close()  


# Example 6/7 - event analysis (read tree, histogramming, fit)

In [ ]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-

from ROOT import *
from array import array
import math

gROOT.ProcessLine(
 "struct Vertex_t {\
    Int_t id;\
    Double_t x,y,z; \
    double perp() { return sqrt(x*x+y*y); } \
};")
pid=array('i',[0])
myVert=Vertex_t()

myChain=TChain('myTree')
myChain.Add('myTree.root')
myChain.SetBranchAddress("vertex",myVert)
myChain.SetBranchAddress("pid",pid)

cFitG = TCanvas('cFitG','cFitG',1000,500)
hDistZ = TH1D('hDistZ','hDistZ',100, 0., 20.)


nEntries = myChain.GetEntries()
for iev in range (nEntries) :
  myChain.GetEntry(iev)
  hDistZ.Fill(myVert.z)
#  print ('pid:{0:2d}, vertex id:{1:3d}, perp={2:5.2f}'.format(pid[0],myVert.id,myVert.perp()))
print ('entires:',nEntries)

hDistZ.DrawCopy()
g1 = TF1("g1", "gaus",-5,5 )
sigmaGauss = 1.
g1.SetParameter(0, 1./sigmaGauss/math.sqrt(2*math.pi) )
g1.SetParameter(1,5.)
g1.SetParameter(2,sigmaGauss)

#
# fit histogram here
#
hDistZ.Fit(g1)
hDistZ.DrawCopy('same')

cFitG.Draw()


# Example 7 - Fitting line

In [ ]:
#!/usr/bin/python3

import sys
import math
import numpy as np
#from array import array
from ROOT import *

def average_meas(points) :
  average = 0.
  for val in  points:
    average += val
  average /= len(points)
  return average

def sigma_meas(points) :
  average = average_meas(points)
  ss=0.
  for point in points: ss+= pow(point-average,2);
  sigma = math.sqrt(ss/(len(points)-1))
  return sigma


x = array('d',[ 1,  3,   4,   5,    6,   7,   8])
y = array('d',len(x)*[0.])
e = array('d',len(x)*[0.])
a = 1.
b = 2.
s = [ 1., 2., 3., 1., 2.,  3., 1.]
g = TF1("g","gaus",-20.,30.)
g.SetParameter(0,1.)
gRandom.SetSeed(2022)

for i in range(len(x)):
  mean=a*x[i]+b
  g.SetParameter(1,mean)
  g.SetParameter(2,s[i])
  values = []
  nValues =10
  for j in range(nValues) :
    val = g.GetRandom()
    values.append(val)

  y[i]=average_meas(values)
  e[i]=sigma_meas(values)/math.sqrt(nValues)
#  y[i]=np.mean(values)
#  e[i]=np.std(values)/math.sqrt(nValues-1) 
  print ('point  i:',i,' x=',x[i],' y=',y[i],' error=',e[i])


xnp = np.array(x)
ynp = np.array(y)
enp = np.array(e)

w=1.0/enp**2
S = np.sum(w)
Sx = np.sum(w*xnp)
Sy = np.sum(w*ynp)
Sxx = np.sum(w*xnp*xnp)
Sxy = np.sum(w*xnp*ynp)
  
D = S*Sxx-Sx*Sx
a_fit = (S*Sxy-Sx*Sy)/D
b_fit = (Sxx*Sy-Sx*Sxy)/D

d = (ynp-(a_fit*xnp+b_fit))/enp
chi2 = np.sum(np.square(d))
print("Fitted values:  a = {},  b = {}".format(a_fit,b_fit))
print("fit    errors: sa = {}, sb = {}".format(math.sqrt(S/D),math.sqrt(Sxx/D)))
print ('chi2/NDF=',chi2/(len(x)-2.))

gr = TGraphErrors(len(x), x, y, array('d',len(x)*[0.]), e)
gr.SetMarkerColor(4)
gr.SetMarkerStyle(21)

'''
gr=TGraphErrors(7)

S=0
Sxx=0
Sxy=0
Sx=0
Sy=0

for i in range (len(x)) :
  gr.SetPoint(i, x[i], y[i])
  gr.SetPointError(i, 0.,e[i])

  w_i=1./e[i]/e[i]
  S+=w_i
  Sxx += x[i]*x[i]*w_i
  Sx += x[i]*w_i
  Sxy += x[i]*y[i]*w_i
  Sy += y[i]*w_i

D = S*Sxx-Sx*Sx
a_fit = (S*Sxy-Sx*Sy)/D
b_fit = (Sxx*Sy-Sx*Sxy)/D

chi2 = 0;
for i in range (0,7) :
  d = (y[i]-(a_fit*x[i]+b_fit))/e[i]
  chi2 += d*d
print (' a=',a_fit,' b=',b_fit)
print ('sa=',math.sqrt(S/D),' sb=',math.sqrt(Sxx/D))
print ('chi2/NDF=',chi2/(len(x)-2.))
'''

c = TCanvas('cFitting','cFitting',600,600)
hDummy = TH1D('hDummy','Prosta',100,0.,10.)
hDummy.SetMaximum(12.)
hDummy.SetMinimum(0.)
hDummy.GetXaxis().SetTitle("x")
hDummy.GetYaxis().SetTitle("y")
hDummy.SetStats(0)
hDummy.DrawCopy()
 
gr.Draw(' p')


fline = TF1("fline", "[0]*x+[1]",0.,9.5)
fline.SetParameter(0,a_fit)
fline.SetParameter(1,b_fit)
fline.Draw('same')

#
# two ways of Fitting
#
#gr.Fit('pol1'," ")
#gr.Fit(fline," ")

c.Print('cFitting.png')
hDummy.Delete()

c.Draw()

# Example 8 - not only pyROOT 

In [ ]:
import numpy as np

import hist
from hist import Hist

histo = Hist(hist.axis.Regular(50, -5, 5))

data = np.random.randn(10000)

histo.fill(data)
histo.plot(); 

for aItem in data:
    histo.fill(aItem)
    
histo.plot();    